In [38]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
import matplotlib as plt
import matplotlib.pylab as plt
import seaborn as sns

In [3]:
from datetime import date, timedelta

start_date = date.today() - timedelta(days = 7)
end_date = date.today() - timedelta(days = 1)

print('get search history from', start_date, 'to', end_date)

get search history from 2020-08-06 to 2020-08-12


In [4]:
q_temp = f'''
SELECT s.page_id,
       s.ref_content,
       s.search_term,
       s.user_id
FROM bun_log_db.app_event_type_search s
JOIN
  (SELECT search_term
   FROM bun_log_db.app_event_type_search
   WHERE event_type = 'search'
     AND device_type IN ('a', 'i')
     AND YEAR||MONTH||DAY BETWEEN {start_date.strftime('%Y%m%d')} AND {end_date.strftime('%Y%m%d')}
   GROUP BY 1
   ORDER BY count(*) DESC
   LIMIT 1000) t ON s.search_term = t.search_term
WHERE s.event_type = 'search'
  AND s.device_type IN ('a', 'i')
  AND s.YEAR||s.MONTH||s.DAY BETWEEN {start_date.strftime('%Y%m%d')} AND {end_date.strftime('%Y%m%d')}
'''

q = '''
SELECT s.page_id,
       s.ref_content,
       s.search_term,
       s.user_id,
       CASE
           WHEN u.sex = 1 THEN 'female'
           WHEN u.sex = 2 THEN 'male'
           ELSE 'none'
       END AS sex,
       Coalesce(NULL, Ceil((Extract(DAY
                                    FROM Getdate() - u.birthdate :: timestamptz) / 365.25)), 0) AS age_detail,
       CASE
           WHEN age_detail < 20 THEN 10
           WHEN age_detail >= 20
                AND age_detail < 30 THEN 20
           WHEN age_detail >= 30
                AND age_detail < 40 THEN 30
           ELSE 40
       END AS age
FROM bun_log_db.app_event_type_search s
JOIN
  (SELECT search_term
   FROM bun_log_db.app_event_type_search
   WHERE event_type = 'search'
     AND device_type IN ('a',
                         'i')
     AND YEAR||MONTH||DAY BETWEEN {} AND {}
   GROUP BY 1
   ORDER BY count(*) DESC
   LIMIT 1000) t ON s.search_term = t.search_term
JOIN service1_quicket.user_extra_info u ON s.user_id = u.uid
WHERE s.event_type = 'search'
  AND s.device_type IN ('a',
                        'i')
  AND s.YEAR||s.MONTH||s.DAY BETWEEN {} AND {}
  AND u.is_identification = 1
  AND To_date(u.birthdate, 'YYYY-MM-DD') > '1900-01-01'
'''.format('\'' + start_date.strftime('%Y%m%d') + '\'', '\'' + end_date.strftime('%Y%m%d') + '\'', '\'' + start_date.strftime('%Y%m%d') + '\'', '\'' + end_date.strftime('%Y%m%d') + '\'')

print(q)
search = pd.read_sql(q, con=bun_dw)


SELECT s.page_id,
       s.ref_content,
       s.search_term,
       s.user_id,
       CASE
           WHEN u.sex = 1 THEN 'female'
           WHEN u.sex = 2 THEN 'male'
           ELSE 'none'
       END AS sex,
       Coalesce(NULL, Ceil((Extract(DAY
                                    FROM Getdate() - u.birthdate :: timestamptz) / 365.25)), 0) AS age_detail,
       CASE
           WHEN age_detail < 20 THEN 10
           WHEN age_detail >= 20
                AND age_detail < 30 THEN 20
           WHEN age_detail >= 30
                AND age_detail < 40 THEN 30
           ELSE 40
       END AS age
FROM bun_log_db.app_event_type_search s
JOIN
  (SELECT search_term
   FROM bun_log_db.app_event_type_search
   WHERE event_type = 'search'
     AND device_type IN ('a',
                         'i')
     AND YEAR||MONTH||DAY BETWEEN '20200806' AND '20200812'
   GROUP BY 1
   ORDER BY count(*) DESC
   LIMIT 1000) t ON s.search_term = t.search_term
JOIN service1_quicket.user_extra_info u ON s

In [5]:
search['search_term'].nunique()

998

In [8]:
start_date = date.today() - timedelta(days = 180)

q_temp = '''
SELECT i.viewer_uid,
       i.target_id,
       i.ref_page,
       i.keyword,
       i.seller_uid,
       split_part(i.ad_ref, '_', 4) AS pu_id,
       split_part(i.ad_ref, '_', 5) AS bid,
       p.price,
       p.category_id
FROM item_click_log6 i
JOIN product_info_for_stats p ON i.target_id = p.pid
WHERE i.updated BETWEEN {} AND {}
  AND i.ad_ref LIKE '%power_up%'
'''.format('\'' + str(start_date) + '\'', '\'' + str(end_date) + '\'')

q = '''
SELECT i.keyword,
       split_part(i.ad_ref, '_', 5) AS bid
FROM item_click_log6 i
WHERE i.updated BETWEEN {} AND {}
  AND i.ad_ref LIKE {}
GROUP BY 1,
         2
'''.format('\'' + str(start_date) + '\'', '\'' + str(end_date) + '\'', '\''+ '%power_up%' + '\'')
print(q)
# power_up_bid = pd.read_sql(q, con=old_dw)


SELECT i.keyword,
       split_part(i.ad_ref, '_', 5) AS bid
FROM item_click_log6 i
WHERE i.updated BETWEEN '2020-02-15' AND '2020-08-12'
  AND i.ad_ref LIKE '%power_up%'
GROUP BY 1,
         2



In [17]:
power_up_bid = pd.read_csv('power_up_bid.csv')
power_up_bid.head()

,keyword,bid
0,발렌티노,150
1,NaN,91
2,컴퓨터본체,83
3,rp충전,140
4,컴퓨터,87


In [10]:
search.head()

,page_id,ref_content,search_term,user_id,sex,age_detail,age
0,검색,최근검색어,신세계,1197297,female,27.0,20
1,검색,최근검색어,스마트워치,4170876,male,44.0,40
2,검색,최근검색어,가내스라,5577231,female,23.0,20
3,검색,None,에어팟,2398586,male,27.0,20
4,검색,최근검색어,네일,4386783,female,18.0,10


In [12]:
search['user_id'].nunique()

196427

In [18]:
search.columns = ['page_id', 'ref_content', 'keyword', 'user_id', 'sex', 'age_detail', 'age']
df = pd.merge(power_up_bid, search, on = 'keyword', how = 'right')

In [19]:
df['bid'] = df['bid'].fillna(0)

In [91]:
# search['keyword'] = search['keyword'].fillna('')
# search['uid'] = search['uid'].astype(str)
# power_up_bid['keyword'] = power_up_bid['keyword'].fillna('')
# power_up_bid['uid'] = power_up_bid['uid'].astype(str)


# df = pd.merge(power_up_bid, search, on = ['keyword', 'uid'])

In [121]:
df[~df['bid'].isna()]['keyword'].nunique()

500

In [124]:
df.groupby('keyword').agg({'bid':lambda x: list(set(x))})

,bid
keyword,
125cc,[0.0]
14k,"[130.0, 132.0, 134.0, 145.0, 150.0, 152.0, 50...."
14k목걸이,"[100.0, 122.0, 134.0, 105.0, 110.0, 80.0, 81.0..."
14k반지,"[130.0, 100.0, 134.0, 105.0, 80.0, 81.0, 50.0,..."
14k팔찌,"[98.0, 105.0, 80.0, 81.0, 50.0, 83.0, 93.0, 82..."
18k,[0.0]
18k팔찌,"[100.0, 135.0, 105.0, 110.0, 80.0, 112.0, 81.0..."
260,"[120.0, 290.0, 80.0, 150.0]"
270,"[80.0, 140.0, 260.0]"


In [21]:
df.groupby('keyword', as_index=False)['bid'].mean().sort_values('bid', ascending=False)

,keyword,bid
583,신발,536.263492
543,스니커즈,526.474576
179,나이키,508.257377
631,아이폰,481.794872
141,구찌,477.067019
424,발렌시아가,422.918860
820,중고폰,417.578450
341,리본돌,410.000000
192,나이키 신발,405.903614
744,운동화,396.309160


In [ ]:
start_date = date.today() - timedelta(days = 7)
end_date = date.today()

'''
SELECT i.keyword,
	   split_part(i.ad_ref, '_', 5) AS bid,
	   split_part(i.ad_ref, '_', 4) AS pu_id,
       i.viewer_uid,
       u.bizlicense,
       CASE
           WHEN u.sex = 1 THEN 'female'
           WHEN u.sex = 2 THEN 'male'
           ELSE 'none'
       END AS sex,
       Coalesce(NULL, Ceil((Extract(DAY
                                    FROM Getdate() - u.birth_date :: timestamptz) / 365.25)), 0) AS age_detail,
       CASE
           WHEN age_detail < 20 THEN 10
           WHEN age_detail >= 20
                AND age_detail < 30 THEN 20
           WHEN age_detail >= 30
                AND age_detail < 40 THEN 30
           ELSE 40
       END AS age
FROM item_click_log6 i
JOIN
  (SELECT keyword
   FROM item_click_log6
   WHERE updated BETWEEN {} AND {}
     AND ad_ref LIKE '%power_up%'
   GROUP BY keyword
   ORDER BY count(*) DESC
   LIMIT 1000) k ON i.keyword = k.keyword
JOIN user_for_stats u ON i.viewer_uid = u.uid
WHERE i.updated BETWEEN {} AND {}
  AND i.ad_ref LIKE '%power_up%'
  AND u.status = 0
  AND To_date(u.birth_date, 'YYYY-MM-DD') > '1900-01-01'
GROUP BY 1, 2, 3, 4, 5, 6, 7, 8;
'''.format('\'' + str(start_date) + '\'', '\'' + str(end_date) + '\'', '\'' + str(start_date) + '\'', '\'' + str(end_date) + '\'')

In [74]:
keyword_biz = pd.read_csv('keyword_biz.csv')

In [75]:
keyword_biz.shape

(169516, 8)

In [76]:
keyword_biz = keyword_biz[~keyword_biz['keyword'].isnull()]

In [77]:
keyword_biz.shape

(83565, 8)

In [78]:
keyword_biz['keyword'].nunique()

998

In [79]:
pu_ids = ','.join('\'' + str(i) + '\'' for i in keyword_biz['pu_id'].unique())

q = '''
SELECT p.pu_id,
       b.ad_start_at,
       b.ad_end_at,
       avg(b.pay_sum) AS pay_sum,
       avg(b.pay_point) AS pay_point,
       avg(b.pay_free) AS pay_free,
       avg(b.max_cpc) AS max_cpc,
       avg(b.pay_limit) AS pay_limit,
       sum(p.stat_view) AS stat_view,
       sum(p.stat_click) AS stat_click
FROM service1_quicket.ad_power_up_point p
JOIN service1_quicket.ad_power_up_budget b ON p.pu_id = b.pu_id
WHERE p.pu_id IN ({})
  AND p.status < 3
  AND p.ad_date BETWEEN {} AND {}
GROUP BY 1,
         2,
         3
'''.format(pu_ids, '\'' + str(start_date) + '\'', '\'' + str(end_date) + '\'')

# AND b.accepted != 2 넣을지 말지
stat = pd.read_sql(q, con=bun_dw)

In [82]:
keyword_biz['pu_id'] = keyword_biz['pu_id'].astype(str)
stat['pu_id'] = stat['pu_id'].astype(str)
data = pd.merge(keyword_biz, stat, on='pu_id')

In [87]:
keyword_biz.head()

,keyword,bid,pu_id,viewer_uid,bizlicense,sex,age_detail,age
0,캠핑,80,100096,215,0,male,36,30
1,본체,200,100238,5282,0,female,27,20
2,컴퓨터 본체,314,100338,5282,0,female,27,20
6,usb,80,99891,13085,0,male,54,40
9,인스타,184,99857,24010,1,male,28,20


In [88]:
stat.head()

,pu_id,ad_start_at,ad_end_at,pay_sum,pay_point,pay_free,max_cpc,pay_limit,stat_view,stat_click
0,99111,2020-08-04 10:49:03,2020-08-06 02:09:03,0,0,0,81,0,15137,514
1,99447,2020-08-06 06:44:05,2020-08-06 08:25:23,0,0,0,100,0,435,18
2,99439,2020-08-06 02:56:05,2020-08-06 09:21:40,0,0,0,80,0,1204,43
3,99455,2020-08-06 08:49:03,2020-08-06 13:41:03,0,0,0,80,0,1925,151
4,99519,2020-08-06 14:14:05,2020-08-07 00:50:03,0,0,0,80,0,1795,59


In [84]:
data.head()

,keyword,bid,pu_id,viewer_uid,bizlicense,sex,age_detail,age,ad_start_at,ad_end_at,pay_sum,pay_point,pay_free,max_cpc,pay_limit,stat_view,stat_click
0,캠핑,80,100096,215,0,male,36,30,2020-08-10 12:32:02,2020-08-14 00:50:06,0,0,0,80,0,10413,78
1,인테리어,80,100096,1545696,0,male,27,20,2020-08-10 12:32:02,2020-08-14 00:50:06,0,0,0,80,0,10413,78
2,캠핑,80,100096,9689010,0,male,46,40,2020-08-10 12:32:02,2020-08-14 00:50:06,0,0,0,80,0,10413,78
3,캠핑,80,100096,773905,0,female,27,20,2020-08-10 12:32:02,2020-08-14 00:50:06,0,0,0,80,0,10413,78
4,캠핑,80,100096,10427616,0,female,32,30,2020-08-10 12:32:02,2020-08-14 00:50:06,0,0,0,80,0,10413,78


In [86]:
data.groupby('keyword', as_index=False)['viewer_uid'].count().sort_values('viewer_uid', ascending=False)

,keyword,viewer_uid
895,컴퓨터,6651
639,아이폰,3165
683,아이폰11,1864
878,카드지갑,1656
304,다이어트,1587
460,문상,1403
143,게이밍컴퓨터,1320
720,아이폰xs,1305
286,노트북,1299
920,클러치,1283


In [108]:
start_date = date.today() - timedelta(days = 7)
end_date = date.today()


print(start_date)
print(end_date)


q = '''
SELECT i.keyword,
       split_part(i.ad_ref, '_', 5) AS bid,
       split_part(i.ad_ref, '_', 4) AS pu_id
FROM item_click_log6 i
JOIN
  (SELECT keyword
   FROM item_click_log6
   WHERE updated BETWEEN {} AND {}
     AND ad_ref LIKE '%power_up%'
   GROUP BY keyword
   ORDER BY count(*) DESC
   LIMIT 1000) k ON i.keyword = k.keyword
WHERE i.updated BETWEEN {} AND {}
  AND i.ad_ref LIKE {}
GROUP BY 1,
         2,
         3
'''.format('\'' + str(start_date) + '\'', '\'' + str(end_date) + '\'', '\'' + str(start_date) + '\'', '\'' + str(end_date) + '\'', '\'' + '%power_up%' + '\'')

print(q)
# keyword_biz = pd.read_sql(q, con=old_dw)

2020-08-07
2020-08-14

SELECT i.keyword,
       split_part(i.ad_ref, '_', 5) AS bid,
       split_part(i.ad_ref, '_', 4) AS pu_id
FROM item_click_log6 i
JOIN
  (SELECT keyword
   FROM item_click_log6
   WHERE updated BETWEEN '2020-08-07' AND '2020-08-14'
     AND ad_ref LIKE '%power_up%'
   GROUP BY keyword
   ORDER BY count(*) DESC
   LIMIT 1000) k ON i.keyword = k.keyword
WHERE i.updated BETWEEN '2020-08-07' AND '2020-08-14'
  AND i.ad_ref LIKE '%power_up%'
GROUP BY 1,
         2,
         3



In [113]:
keyword_biz = pd.read_csv('keyword_biz_2.csv')
keyword_biz = keyword_biz[~keyword_biz['keyword'].isnull()]
keyword_biz['keyword'].nunique()

999

In [114]:
pu_ids = ','.join('\'' + str(i) + '\'' for i in keyword_biz['pu_id'].unique())

q = '''
SELECT v.user_id,
       v.ref_term,
       v.ref_campaign AS pu_id,
       CASE
           WHEN u.sex = 1 THEN 'female'
           WHEN u.sex = 2 THEN 'male'
           ELSE 'none'
       END AS sex,
       Coalesce(NULL, Ceil((Extract(DAY
                                    FROM Getdate() - u.birthdate :: timestamptz) / 365.25)), 0) AS age_detail,
       CASE
           WHEN age_detail < 20 THEN 10
           WHEN age_detail >= 20
                AND age_detail < 30 THEN 20
           WHEN age_detail >= 30
                AND age_detail < 40 THEN 30
           ELSE 40
       END AS age
FROM bun_log_db.app_event_type_view v
JOIN service1_quicket.user_extra_info u ON v.user_id = u.uid
WHERE v.year||v.month||v.day BETWEEN {} AND {}
  AND ref_campaign IN ({})
  AND ref_source = 'power_up'
  AND event_type = 'view'
  AND To_date(u.birthdate, 'YYYY-MM-DD') > '1900-01-01'
'''.format('\'' + start_date.strftime('%Y%m%d') + '\'', '\'' + end_date.strftime('%Y%m%d') + '\'', pu_ids)

imp_df = pd.read_sql(q, con=bun_dw)

In [116]:
print(imp_df.shape)
imp_df.head()

(167015, 6)


,user_id,ref_term,pu_id,sex,age_detail,age
0,7890366,None,99844,male,27.0,20
1,8273901,None,100412,male,25.0,20
2,4641952,아이폰 xr,100292,male,22.0,20
3,8273901,None,100078,male,25.0,20
4,11396070,제작,100361,female,16.0,10


In [117]:
keyword_biz.head()

,keyword,bid,pu_id
0,아이패드 에어3,80,99621
1,노트북,98,99551
5,아이폰8 플러스,80,98800
6,별풍선,100,99484
7,인스타,200,99356


In [ ]:
"""
SELECT split_part(i.ad_ref, '_', 4) AS pu_id,
       split_part(i.ad_ref, '_', 5) AS bid,
       i.target_id
FROM item_click_log6 i
WHERE i.updated BETWEEN '2020-08-07' AND '2020-08-14'
  AND i.ad_ref LIKE '%power_up%'
GROUP BY 1,
         2,
         3;
"""

In [119]:
power_up = pd.read_csv('power_up.csv')
power_up.head()

,pu_id,bid,target_id
0,99493,80,130346965
1,99549,80,129938197
2,99099,50,22143320
3,93820,141,128501885
4,97976,191,129987745


In [122]:
pids = ','.join('\'' + str(i) + '\'' for i in power_up['target_id'].unique())

q = '''
WITH t AS
  (SELECT id AS pid,
          keywor
   FROM service1_quicket.product_info
   WHERE id IN ({}))
  (SELECT pid,
          split_part(keyword, ',', 1) AS keyword
   FROM t)
UNION
  (SELECT pid,
          split_part(keyword, ',', 2) AS keyword
   FROM t)
UNION
  (SELECT pid,
          split_part(keyword, ',', 3) AS keyword
   FROM t)
UNION
  (SELECT pid,
          split_part(keyword, ',', 4) AS keyword
   FROM t)
UNION
  (SELECT pid,
          split_part(keyword, ',', 5) AS keyword
   FROM t)
'''.format(pids)

keyword = pd.read_sql(q, con=bun_dw)

In [151]:
pu_ids = ','.join('\'' + str(i) + '\'' for i in power_up['pu_id'].unique())

q = '''
SELECT v.imp_id,
       v.ref_term,
       v.ref_campaign AS pu_id,
       CASE
           WHEN u.sex = 1 THEN 'female'
           WHEN u.sex = 2 THEN 'male'
           ELSE 'none'
       END AS sex,
       Coalesce(NULL, Ceil((Extract(DAY
                                    FROM Getdate() - u.birthdate :: timestamptz) / 365.25)), 0) AS age_detail,
       CASE
           WHEN age_detail < 20 THEN 10
           WHEN age_detail >= 20
                AND age_detail < 30 THEN 20
           WHEN age_detail >= 30
                AND age_detail < 40 THEN 30
           ELSE 40
       END AS age
FROM bun_log_db.app_event_type_view v
JOIN service1_quicket.user_extra_info u ON v.user_id = u.uid
  AND ref_campaign IN ({})
  AND ref_source = 'power_up'
  AND event_type = 'view'
  AND To_date(u.birthdate, 'YYYY-MM-DD') > '1900-01-01'
'''.format(pu_ids)

imp_df = pd.read_sql(q, con=bun_dw)

In [153]:
imp_df.shape

(289621, 6)

In [131]:
print(power_up.columns)
print(keyword.columns)
print(imp_df.columns)

Index(['pu_id', 'bid', 'target_id'], dtype='object')
Index(['pid', 'keyword'], dtype='object')
Index(['imp_id', 'ref_term', 'pu_id', 'sex', 'age_detail', 'age'], dtype='object')


In [154]:
power_up.columns = ['pu_id', 'bid', 'pid']
power_up['pid'] = power_up['pid'].astype(str)
keyword['pid'] = keyword['pid'].astype(str)
power_up['pu_id'] = power_up['pu_id'].astype(str)
imp_df['pu_id'] = imp_df['pu_id'].astype(str)

In [155]:
df = pd.merge(power_up, keyword, on='pid')
df = pd.merge(df, imp_df, on='pu_id')
df.head()

,pu_id,bid,pid,keyword,imp_id,ref_term,sex,age_detail,age
0,99493,80,130346965,아이폰SE2,99e35f2c640e02f891a5,아이폰 se,male,26.0,20
1,99493,80,130346965,아이폰SE2,970a5f2c6adf030d6a81,아이폰8,female,18.0,10
2,99493,80,130346965,아이폰SE2,99315f2c6c71030bf9ea,아이폰se2,male,24.0,20
3,99493,80,130346965,아이폰SE2,99315f2c6a810308bbb0,아이폰8,male,18.0,10
4,99493,80,130346965,아이폰SE2,99e35f2c64fe02fa48af,아이폰se,female,18.0,10


In [156]:
df.shape

(6950783, 9)

In [159]:
df['keyword'].unique()

array(['아이폰SE2', '아이폰SE', '아이폰7+', ..., '배기스5인치', '우븐반바지', '우븐플로우'],
      dtype=object)

In [ ]:
df['keyword'] = df['keyword'].str.lower()

In [158]:
# app_event_type_view 를 일주일로 리밋을 두었을 때
# df.to_csv('final_df.csv', index=False) 
data = pd.read_csv('final_df.csv')
data.shape

(5345154, 9)

In [137]:
df.columns

Index(['pu_id', 'bid', 'pid', 'keyword', 'imp_id', 'ref_term', 'sex',
       'age_detail', 'age'],
      dtype='object')

In [139]:
df.groupby('keyword', as_index=False).agg({'bid':lambda x: list(set(x))})

,keyword,bid
0,,"[50, 80, 81, 600, 90, 91, 98, 100, 101, 104, 1..."
1,55,"[80, 50]"
2,마네킹,"[80, 50]"
3,여성,"[80, 50]"
4,#10세대 #컴퓨터 #중고컴퓨터 #배그 #배그용컴퓨터,[80]
5,#롤 #롤강의 #롤듀오 #롤배치,[80]
6,#롤강의,"[160, 200, 170, 140, 80, 210, 50, 180, 120]"
7,#롤듀오,"[160, 200, 170, 140, 80, 210, 50, 180, 120]"
8,#롤배치,"[160, 200, 170, 140, 80, 210, 50, 180, 120]"
9,00198,"[80, 50]"
